In [7]:
# base
import pandas as pd
import os, time
from multiprocessing import Pool, cpu_count

# my function
from mosaic_rasters_functions import summarize_structure

In [8]:
# generate list of csv files to iterate through
csv_dir = os.path.join("..", "data", "all_vars", "sampled")
csv_list = []
for file in os.listdir(csv_dir):
    if file.endswith(".csv"):
        csv_list.append(os.path.join(csv_dir, file))

In [10]:
start_time = time.time()
if __name__ == '__main__':
    with Pool() as pool:
        out_dfs = pool.map(summarize_structure, csv_list)
        
pd.concat(out_dfs).to_csv(os.path.join("..", "data", "structure_means.csv"))
print(time.time() - start_time)

59.80458617210388
